<a href="https://colab.research.google.com/github/SS-2005/Fake_Review_Detector/blob/main/Fake_Review_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Define your neural network architecture
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Custom dataset class
class ReviewDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Load your dataset
data = pd.read_csv('/content/fake reviews dataset.csv')
# Encode labels as 0 and 1
data['label'] = data['label'].apply(lambda x: 0 if x == 'CG' else 1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['text_'], data['label'], test_size=0.2, random_state=42)

# Convert text data into numerical vectors using CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define hyperparameters
input_size = X_train.shape[1]
hidden_size = 128
output_size = 1
learning_rate = 3e-4
num_epochs = 10
batch_size = 32

# Create DataLoader for batch processing
train_dataset = ReviewDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model
model = BinaryClassifier(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 == 0:
            predicted = (outputs > 0.5).float()
            accuracy = (predicted == labels.unsqueeze(1)).float().mean()
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f} Accuracy: {accuracy.item()*100:.2f}%')

# Evaluation
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted = (outputs > 0.5).float()
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')

Epoch [1/10], Step [100/1011], Loss: 0.4333 Accuracy: 75.00%
Epoch [1/10], Step [200/1011], Loss: 0.3648 Accuracy: 87.50%
Epoch [1/10], Step [300/1011], Loss: 0.1722 Accuracy: 100.00%
Epoch [1/10], Step [400/1011], Loss: 0.2066 Accuracy: 90.62%
Epoch [1/10], Step [500/1011], Loss: 0.2579 Accuracy: 90.62%
Epoch [1/10], Step [600/1011], Loss: 0.3281 Accuracy: 84.38%
Epoch [1/10], Step [700/1011], Loss: 0.3293 Accuracy: 84.38%
Epoch [1/10], Step [800/1011], Loss: 0.2535 Accuracy: 84.38%
Epoch [1/10], Step [900/1011], Loss: 0.2017 Accuracy: 93.75%
Epoch [1/10], Step [1000/1011], Loss: 0.2599 Accuracy: 87.50%
Epoch [2/10], Step [100/1011], Loss: 0.1649 Accuracy: 87.50%
Epoch [2/10], Step [200/1011], Loss: 0.1349 Accuracy: 96.88%
Epoch [2/10], Step [300/1011], Loss: 0.1637 Accuracy: 93.75%
Epoch [2/10], Step [400/1011], Loss: 0.0848 Accuracy: 96.88%
Epoch [2/10], Step [500/1011], Loss: 0.2540 Accuracy: 93.75%
Epoch [2/10], Step [600/1011], Loss: 0.2324 Accuracy: 87.50%
Epoch [2/10], Step [70

In [4]:
# Save the model
torch.save(model.state_dict(), 'fakeReviewsClassifier.pth')

In [5]:
import pickle

with open("count_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [6]:
# Reload vectorizer
with open("count_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)


In [7]:
# Recreate model
model = BinaryClassifier(
    input_size=len(vectorizer.get_feature_names_out()),
    hidden_size=128,
    output_size=1
)

model.load_state_dict(torch.load("fakeReviews_classifier.pth", map_location="cpu"))
model.eval()


BinaryClassifier(
  (fc1): Linear(in_features=36975, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

In [8]:
def predict_review(text):
    text_vector = vectorizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    with torch.no_grad():
        logits = model(text_tensor)
        probability = torch.sigmoid(logits).item()

    if probability >= 0.5:
        return "GENUINE REVIEW", round(probability * 100, 2)
    else:
        return "FAKE REVIEW", round((1 - probability) * 100, 2)


In [17]:
#Sinlge Comment
comment = "I was a little apprehensive about this one, due to the the comments here. I took the plunge after confirming that it would work with my HP 6540 printer. Setup was easy - but since I run WPA on the wireless side, hooking up to the wired side was a necessity"

label, confidence = predict_review(comment)

print("Review:", comment)
print("Prediction:", label)
print("Confidence:", confidence, "%")


Review: I was a little apprehensive about this one, due to the the comments here. I took the plunge after confirming that it would work with my HP 6540 printer. Setup was easy - but since I run WPA on the wireless side, hooking up to the wired side was a necessity
Prediction: GENUINE REVIEW
Confidence: 100.0 %


In [10]:
#multiple comments
comments = [
    "Best product ever buy now buy now!!!",
    "Very satisfied with the quality and delivery",
    "Limited time offer!!! Click the link!!!"
]

for c in comments:
    label, confidence = predict_review(c)
    print(f"{c} --> {label} ({confidence}%)")


Best product ever buy now buy now!!! --> GENUINE REVIEW (97.9%)
Very satisfied with the quality and delivery --> GENUINE REVIEW (84.64%)
Limited time offer!!! Click the link!!! --> GENUINE REVIEW (97.68%)
